In [7]:
import pandas as pd
import pydeck
import psycopg2 as pg
from psycopg2.extras import DictCursor
import geopandas as gpd
import h3pandas

import configparser

config = configparser.ConfigParser()
config.read("../../config/config.ini")    
db_params = dict(config['DB'])
from sqlalchemy import create_engine

In [6]:
sql = """
WITH  total_working_pop as 
	(SELECT SUM(population) as city_pop
	FROM h3demographics 
	JOIN cityh3map ON h3demographics.h3id = cityh3map.h3id
	WHERE
	h3demographics.categorytype = 'Age and Sex'
	AND h3demographics.groupname IN ('18 to 45 years male', '18 to 45 years female', '45 to 65 years male', '45 to 65 years female')	
	AND cityh3map.cityid = %s)

SELECT *, pop / city_pop as ratio FROM (
	select cityh3map.h3id, SUM(catchment_stats.population) as pop from cityh3map
	JOIN catchments 
	ON catchments.originh3id = cityh3map.h3id 
	JOIN catchment_stats ON catchment_stats.catchmentid = catchments.catchmentid
	WHERE cityh3map.cityid = %s AND catchment_stats.categorytype = 'Age and Sex' AND catchments.timeofday = 'morning'
	AND catchment_stats.groupname IN ('18 to 45 years male', '18 to 45 years female', '45 to 65 years male', '45 to 65 years female')
	GROUP BY cityh3map.h3id, catchments.catchmentid) as a
	CROSS JOIN total_working_pop
	ORDER BY pop DESC
"""

with pg.connect(**db_params) as conn:
    with conn.cursor(cursor_factory=DictCursor) as cur:
        cur.execute(sql, (1, 1))   
        data = cur.fetchall()

In [5]:
data

[['8944c1a8eabffff',
  1044,
  116411.37059307197,
  1667771.0000000102,
  0.06980057249650656],
 ['8944c1a8133ffff',
  4,
  103434.91115068384,
  1667771.0000000102,
  0.06201985233625192],
 ['8944c1a8e2fffff',
  804,
  102096.6898327489,
  1667771.0000000102,
  0.06121745121647293],
 ['8944c1a813bffff',
  71,
  101862.48011516589,
  1667771.0000000102,
  0.06107701843668301],
 ['8944c1a812bffff',
  923,
  100614.14575824255,
  1667771.0000000102,
  0.060328513781713396],
 ['8944c1a8e07ffff',
  906,
  95751.9378685564,
  1667771.0000000102,
  0.05741312078729982],
 ['8944c1a8ab3ffff',
  1076,
  92045.99169333758,
  1667771.0000000102,
  0.05519102544254398],
 ['8944c1aab5bffff',
  370,
  89194.41877923493,
  1667771.0000000102,
  0.0534812146147369],
 ['8944c1a810fffff',
  1034,
  89173.08514199172,
  1667771.0000000102,
  0.053468422908175745],
 ['8944c1a8123ffff',
  924,
  88368.70660329748,
  1667771.0000000102,
  0.05298611536193935],
 ['8944c132093ffff',
  892,
  87907.8922681233

In [ ]:
sql = """
select h3id, population FROM h3demographics WHERE
groupname = 'Native Hawaiian and Other Pacific Islander' and population > 0
"""

In [14]:
view_state = pydeck.ViewState(
    **{"latitude": 33.7, "longitude": -84.3, "zoom": 10, "maxZoom": 16, "pitch": 0, "bearing": 0}
)

h3_layer = pydeck.Layer(    
    "H3HexagonLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon="h3id",    
    get_line_color=[0, 0, 0],
    get_fill_color=[100, 100, 100],
    line_width_min_pixels=2,
)

r = pydeck.Deck(
    [h3_layer],
    initial_view_state=view_state,    
    map_style=pydeck.map_styles.LIGHT,
)
r.to_html()